In [2]:
import pandas as pd

# Load the data
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# Show top rows
train_df.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# Combine datasets for consistent preprocessing
full_df = pd.concat([train_df, test_df], sort=False)

# Fill missing Age with median
full_df["Age"].fillna(full_df["Age"].median(), inplace=True)

# Fill Embarked with mode
full_df["Embarked"].fillna(full_df["Embarked"].mode()[0], inplace=True)

# Fill Fare with median (for test set)
full_df["Fare"].fillna(full_df["Fare"].median(), inplace=True)

# Convert Sex and Embarked to numeric
full_df["Sex"] = full_df["Sex"].map({"male": 0, "female": 1})
full_df["Embarked"] = full_df["Embarked"].map({"S": 0, "C": 1, "Q": 2})

# Drop unused columns
full_df.drop(columns=["Cabin", "Ticket", "Name", "PassengerId"], inplace=True)

# Split back into train and test
X_train = full_df[:len(train_df)].drop("Survived", axis=1)
y_train = train_df["Survived"]
X_test = full_df[len(train_df):].drop("Survived", axis=1)


C:\Users\DELL\AppData\Local\Temp\ipykernel_13868\530657693.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  full_df["Age"].fillna(full_df["Age"].median(), inplace=True)
C:\Users\DELL\AppData\Local\Temp\ipykernel_13868\530657693.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.


In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Split into train/validation to evaluate locally
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Train model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_tr, y_tr)

# Evaluate
val_pred = model.predict(X_val)
print("Validation accuracy:", accuracy_score(y_val, val_pred))


Validation accuracy: 0.8268156424581006


In [8]:
predictions = model.predict(X_test)

# Create submission file
submission = pd.DataFrame({
    "PassengerId": test_df["PassengerId"],
    "Survived": predictions
})
submission.to_csv("Survived.csv", index=False)
